# Using Metacritic and HowLongToBeat data to determine which games from my backlog are worth playing

This is a personal problem that every gamer faces at some point. Currently I own over 500  video games on 18 different platforms and storefronts in both physical and digital. Picking out a new game to play is becoming a daunting task so I've decided to use data from Metacritic, mainly Critic reviews, and the number of said reviews as well as average length of a playthrough as provided by HowLongToBeat to determine which of my games are worth playing.

In [1]:
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
from howlongtobeatpy import HowLongToBeat as hltb

## Data

### Preparing and reading data

In [2]:
list_of_games = pd.read_csv(r"data/game_list.csv", encoding='cp1252')

I've made a list of every game I own in .csv format. The document is pretty simple it contains the name of the game in the first column and the platform for which I own the game in the second. All my Steam, Epic Store, Origin, Amazon and other PC storefronts were marked as 'pc'.

Example of the .csv file:

| name | platform |
| --- | --- |
| Abzu | playstation-4 |
| Alien Isolation | playstation-3 |
| Animal Crossing Amiibo Festival | wii-u |
| Animal Crossing New Horizons | switch |
| Assassin's Creed | pc |

### Getting Metacritic data

I want to have the metacritic critics score as well as number of reviews that match the platform I own the game on. However matacritic does not provide any api, or database for their game information, so I've decided to scrape metacritic to get the data I need.

Page for every game contains it's 'Metascore', the critic aggregate review score as well as number of said reviews. Using page inspection tool I've detemined which html code blocks have the data I need. I will feed the tags into the html parser and get the necessary data. For example here is the game Enter the Gungeon for Playstation 4 <img src="images/metacritic_example.png" alt="Metacritic example" />. The Metascore is in this html element:
```html
<div class="metascore_w xlarge game positive">
    <meta itemprop="worstRating" content="0">
    <meta itemprop="bestRating" content="100">
    <span itemprop="ratingValue">82</span>
</div>
```

Metacritic generates the link to the page of a game using 'https://www.metacritic.com/game/' base url then adding the platform for example 'playstation-4' and then the game name hyphenated and in lowercase, e.g. enter-the-gungeon, making the final url https://www.metacritic.com/game/playstation-4/enter-the-gungeon which is perfect, because I have an easy way to generate the url to grab the html, and then parse out the data I need.

In [3]:
def extract_mc_data(series):
    url = 'https://www.metacritic.com/game/'
    name = series['name'].replace('.','').replace(',','').replace('-','').replace('\'','').replace(':','').replace(' ','-')
    #removing any symbols from the game title that do not appear in metacritic links,
    #replacing any spaces with '-'
    session = requests.Session()
    session.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    #Making sure that requests are sent as if they were done through a browser,
    #otherwise metacritic does not return the correct page.
    full_url = url + series['platform'] + '/' + name.lower() + '/'
    #creating the metacritic link, making sure name is in all lowercase as the metacritic links are case-sensitive
    response = session.get(full_url)
    parser = BeautifulSoup(response.content, 'html.parser')
    #dowloading the full html page for each game and feeding it into a html parser
    
    review_count_elements = parser.find("span", class_="count")
    if review_count_elements is not None:
        review_count_text = review_count_elements.text.strip()
        review_count = int(re.findall(r'[\d]+', review_count_text)[0])
    else:
        review_count = 0
    #Checking if the html file contains text 'based on x Critic Reviews'
    #if it does then extraxting the number of reviews using a simple regex and converting to int
    #if game contains 0 reviews, or page for a game does not exist then review count is set to 0
    
    score_elements = parser.find("span", itemprop="ratingValue")
    if score_elements is not None:
        score_text = score_elements.text.strip()
        score = int(score_text)
    else:
        score = 0
    #Checking if the html file contains element 'ratingValue'
    #if it does then extraxting the critics score and converting to int
    #if game contains 0 reviews, or page for a game does not exist then critics score is set to 0
    return pd.Series([series['name'], series['platform'], score, review_count], index =['name', 'platform', 'score', 'total_reviews'])

In [4]:
print(extract_mc_data(list_of_games.iloc[34]))
#testing to see if function works

name             Enter the Gungeon
platform             playstation-4
score                           82
total_reviews                   33
dtype: object


In [5]:
subset_10 = list_of_games.head(10)

In [6]:
subset_10 = subset_10.apply(extract_mc_data, axis='columns')

In [7]:
print(subset_10)

                              name       platform  score  total_reviews
0                             Abzu  playstation-4     78             72
1                  Alien Isolation  playstation-3      0              0
2  Animal Crossing Amiibo Festival          wii-u     46             20
3     Animal Crossing New Horizons         switch     90            111
4                 Assassin's Creed             pc      0              0
5                        Bayonetta          wii-u     86             18
6                      Bayonetta 2          wii-u     91             80
7                    Balloon Fight            NES      0              0
8             Batman Arkham Asylum  playstation-3     91             70
9                       BloodBorne  playstation-4     92            100


In [8]:
list_of_games = list_of_games.apply(extract_mc_data, axis='columns')

In [9]:
list_of_games.head(10)

,name,platform,score,total_reviews
0,Abzu,playstation-4,78,72
1,Alien Isolation,playstation-3,0,0
2,Animal Crossing Amiibo Festival,wii-u,46,20
3,Animal Crossing New Horizons,switch,90,111
4,Assassin's Creed,pc,0,0
5,Bayonetta,wii-u,86,18
6,Bayonetta 2,wii-u,91,80
7,Balloon Fight,NES,0,0
8,Batman Arkham Asylum,playstation-3,91,70
9,BloodBorne,playstation-4,92,100


In [10]:
list_of_games.tail(10)

,name,platform,score,total_reviews
516,Burnout Paradise: The Ultimate Box,pc,87,26
517,Peggle,pc,0,0
518,Bejeweled 3,pc,82,27
519,Dragon Age: Origins,pc,91,67
520,Syndicate,pc,69,16
521,Mass Effect 2,pc,94,55
522,Medal of Honor Pacific Assault,pc,80,43
523,SteamWorld Dig,pc,76,12
524,Plants vs. Zombies Game of the Year Edition,pc,0,0
525,The Sims 4,pc,70,75


In [11]:
list_of_games[list_of_games['score'] == 0].head(15)

,name,platform,score,total_reviews
1,Alien Isolation,playstation-3,0,0
4,Assassin's Creed,pc,0,0
7,Balloon Fight,NES,0,0
10,Bubble Bobble,NES,0,0
13,Castlevania,NES,0,0
14,Castlevania II: Simon's Quest,NES,0,0
15,Castlevania: Symphony of the Night,playstation-3,0,0
16,Colin McRae Dirt,playstation-3,0,0
27,Donkey Kong,NES,0,0
28,Donkey Kong Jr.,NES,0,0


Web scraping from metacritic was a success! Now my game list contains both the critics score and the number of reviews. Unfortunately there are a number of games that contain 0 reviews and have a score of 0. In some cases there is simply no page for the game on metacritic. For example I own an original copy of Assassin's Creed on PC however metacritic does not have a page for the original version of the game, and only has a Director's cut re-release page. I would consider these different enough and do not want to use one score to fill in gaps in the other, so I'll leave them as 0 for now. 

Some games metacritic does have a page for, but there are no reviews on the specific version of the game I own. For example Alien Isolation on Playstation 3 contains 0 critic reviews on metacritic.

Another issue is that Metacritic only contains pages and reviews for the games starting with the sixth generation of consoles. Meaning any games I own for the original Playstation, GameBoy, GameBoy Color, Nintendo Entertainment System, or Nintendo 64 would automatically have 0 score and 0 reviews.

### Getting HowLongToBeat Data

Getting data from HowLongToBeat using a web scraper is a lot harder than it is from Metacritic. First of all, how long to beat uses game id to generate link to specific game page. For example game Enter the Gungeon has the id of 35549 https://howlongtobeat.com/game/35549. There is no easy way to get these id or generate them. So using the same method I did for Metacritic is not viable.

The second thing I tried was to use their search functionality and grab the first game from the search to extract the data I need. It appeared that I could generate the search link using the name of the game.
<img src="images/hltb_example.png" alt="HowLongToBeat Search" />
Unfortunately hltb generates their page dynamically, so when the scraper tries to go to https://howlongtobeat.com/?q=Legend%2520of%2520Zelda for example it just downloads the front page, and the search elements are not loaded.

Thankfully I didn't have to deal with complex webpage design for this particular case. As there are several libraries designed to do just what I need. I've chosen [howlongtobeatpy](https://pypi.org/project/howlongtobeatpy/) as it appeared to be an easy-to-use simple library that had everything I needed.

I'll be using the search functionality from the library to get list of games that match search criteria, compare the names, to find an exact match and then extract both 'Main Story' playtime and 'Main Story + Extras' playtime.

In [12]:
def extract_hltb_data(name):
    matching_games = hltb().search(name)
    #Passing in the name of the game to the hltb library
    gametime = [0, 0, 0]
    #setting default gametime values
    if matching_games is not None:
    #checking if any results were returned
        for game in matching_games:
            if name == game.game_name:
    #Looking for an exact match in the returned games
                gametime[0] = game.main_story
                gametime[1] = game.main_extra
                
    gametime[2] = (gametime[0] + gametime[1]) / 2
    #Calculating time for an average playthrough
    return pd.Series(gametime)

### Calculating time for an average playthrough

Besides grabbing time for average completion of Main story, and completing some extras and sidequests I'll be calculating the average of the two. In general I complete the main story and engage with different sidequests and systems of the game but rarely complete them. So this average would be more representative of my potential playthrough than any of the two values separately.

In [13]:
print(extract_hltb_data(list_of_games.iloc[216]['name']))

0    10.84
1    10.94
2    10.89
dtype: float64


In [14]:
list_of_games[['main_story', 'main_extra', 'avg_playthrough']] = list_of_games['name'].apply(extract_hltb_data)

In [15]:
list_of_games.tail(10)

,name,platform,score,total_reviews,main_story,main_extra,avg_playthrough
516,Burnout Paradise: The Ultimate Box,pc,87,26,0.00,0.00,0.000
517,Peggle,pc,0,0,0.00,0.00,0.000
518,Bejeweled 3,pc,82,27,4.23,11.27,7.750
519,Dragon Age: Origins,pc,91,67,40.35,58.42,49.385
520,Syndicate,pc,69,16,12.05,12.22,12.135
521,Mass Effect 2,pc,94,55,24.42,35.47,29.945
522,Medal of Honor Pacific Assault,pc,80,43,0.00,0.00,0.000
523,SteamWorld Dig,pc,76,12,5.06,6.53,5.795
524,Plants vs. Zombies Game of the Year Edition,pc,0,0,0.00,0.00,0.000
525,The Sims 4,pc,70,75,18.16,68.40,43.280


## Working with data

I can't do any calculations or direct comparisons between the three main values I've gathered so far. Score points per hour or similar calculations would not provide us with any valuable insights. As well as adding up or calculating any aggregate of the three as they are. They are all in different scales and measure different things. 
First I need to normalise these values without losing what the data represents. For this I will be using Scaling to Range normalization using this equation:
$$
x`= \frac{x-x_{min}}{x_{max}-x_{min}}\
$$

In [16]:
list_of_games['normalized_score'] = list_of_games['score'].apply(
    lambda x: (x - list_of_games['score'].min())/(list_of_games['score'].max() - list_of_games['score'].min()))

In [17]:
list_of_games['normalized_reviews'] = list_of_games['total_reviews'].apply(
    lambda x: (x - list_of_games['total_reviews'].min())/(list_of_games['total_reviews'].max() - list_of_games['total_reviews'].min()))

In [18]:
list_of_games['normalized_time'] = list_of_games['avg_playthrough'].apply(
    lambda x: (x - list_of_games['avg_playthrough'].min())/(list_of_games['avg_playthrough'].max() - list_of_games['avg_playthrough'].min()))

When normalizing the values I had to come to a decision. I could set the min value when calculating every normalized value to 0, and not look for minimum value. For critics score I could've set the maximum value as 100, as there can't be a higher value than that on metacritic. I've had to decide whether I want to normalize them in relation to each other and keep it in context of the relevant data, or normalize them in relation to general boundries that those values could have.

Since there is no set maximum limit to reviews, or playtime calculating score with a maximum limit (even if no value matches that limit in my dataset) would not be correct in my opinion, and I chose to extract the minimum and maximum values for each of the columns.

#### Calculating entertainment value of each game

Now as all of the necessary values are calculated I can start using them to determine the entertainment value of each game. My first idea was to simply calculate an average of the three. However this simple approach proved to have an interesting problem. Length of the game when normalized favours long games, meaning the longer the game is the higher the normalized value, the more skewed towards longer games the entertainment value score is. And while yes, a 100hr. game with a critic score of 97, would technically have more entertainment value than say 10 hour game which scored a 90, I wouldn't necessarily want to play say 5 100h. or longer games in a row.

A solution for this problem is to give weights to the scores. I've chosen the critics score to be the most important making up 50% of the final value, second most important was number of reviews at 30% value. Number of reviews are important as I feel that game with a score of 99 and only 4 reviews is not as accurately represented as a game with a score of 80, but over 100 reviews. Lastly making up 20% of the final score is the length of the playthrough, and while it is still favouring longer games, this way the impact of length to calculated entertainment value is lessened.

In [26]:
list_of_games['playability_score'] = list_of_games.apply(
    lambda x: round(((x['normalized_score']*0.5)+(x['normalized_reviews']*0.3)+(x['normalized_time']*0.2)) * 100, 2),
    axis='columns')

In [27]:
list_of_games.sort_values(['playability_score'], inplace=True, ascending=False)
list_of_games[['name', 'platform', 'playability_score']].head(15)

,name,platform,playability_score
172,Super Mario Odyssey,switch,78.46
184,The Last Of Us Part II,playstation-4,77.94
53,God Of War,playstation-4,77.71
187,The Legend of Zelda Breath of the Wild,switch,77.02
207,Uncharted 4: A Thief's End,playstation-4,76.33
203,Uncharted 2: Among Thieves,playstation-3,75.78
234,Dota 2,pc,74.57
3,Animal Crossing New Horizons,switch,73.91
183,The Last Of Us,playstation-3,73.27
9,BloodBorne,playstation-4,72.22


In [28]:
list_of_games.to_csv(r"results\backlog.csv")

Finaly the results were sorted based on their entertainment value, or playability_score and saved into a csv document.

Now I can easily choose which game I need to play next. And it looks like I need to play Super Mario Odyssey, which has in fact sat on my shelf for over 4 years now collecting dust.